
***ARPEGGIATOR IMPLEMENTATION***


In [1]:
# imports 
%load_ext autoreload
%autoreload 2
import schillinger.rhythm as SR
import schillinger.pitch as SP
import itertools
import random
import music21
%load_ext music21.ipython21

In [2]:
def showNotes(note_sequences, time_meter):
    TS = str(time_meter[0])+'/'+str(time_meter[1])
    
    s = music21.stream.Score(id='mainScore')
    
    
    for it, seq in enumerate(note_sequences):
        part = music21.stream.Part(id='part'+str(it))
        part.timeSignature = music21.meter.TimeSignature(TS)
        for i, nota in enumerate(seq):
            part.append(nota)
            
        s.insert(0, part)
    s = s.transpose(0)
    return s

In [3]:
def Arpeggiator(chord, duration, speed, arp_type="UP", arp_range=1):
    '''arg1 = music21.chord, arg2=duration of chord, arg3 = speed of arpeggiator..'''
    
    sequence = []
    
    note_duration = int(duration/speed)
    
    note_amount = len(chord)
    seq_layout = []
    for r in range(arp_range):
        for i in chord:
            tn = music21.pitch.Pitch(i.nameWithOctave)
            tn.octave = tn.octave + r
            seq_layout.append(tn)
            
    
    seq_vorwards = seq_layout
    seq_backwards = list(reversed(seq_layout))
    
    
    if arp_type == "DOWN":
        temp_sequence = seq_backwards
    elif arp_type =="UP":
        temp_sequence = seq_vorwards
    elif arp_type=="UPDOWN":
        temp_sequence = seq_vorwards+seq_backwards
    elif arp_type=="UPDOWN2":
        temp_sequence = seq_vorwards+seq_backwards[1:-1]
    elif arp_type=="DOWNUP":
        temp_sequence = seq_backwards+seq_vorwards
    elif arp_type=="DOWNUP2":
        temp_sequence = seq_backwards+seq_vorwards[1:-1]
    
    for i in range(note_duration):
        seq_limit = i%len(temp_sequence)
        pitch = music21.note.Note(temp_sequence[seq_limit].nameWithOctave)
        pitch.duration.quarterLength = speed
        sequence.append(pitch)
    
    return sequence

In [8]:
#merged_harmony = list(itertools.chain(*whole_seq))
chords = [["C4","E-4","G4","B-4","D5","F5"],["G4","B4","D5","F4","A4","C5"]]
arp_seq = []
for i, w in enumerate(chords):
    test_chord = music21.chord.Chord(w)
    #test_chord = test_chord.sortDiatonicAscending()
    #test_chord = test_chord.semiClosedPosition()
    #test_chord = test_chord.closedPosition()
    dur = 4
    speed = .25
    apr_chord = Arpeggiator(test_chord, dur, speed, arp_type="UPDOWN2",arp_range=2)
    arp_seq.append(apr_chord)
    

harmony_seq = list(itertools.chain(*arp_seq)) # convert to [note1[harmony1], ... noteN[harmonyN]]

In [9]:

merged_bass = ["C4","G4"]
bass_seq = []
for i,b in enumerate(merged_bass):
    nota = music21.note.Note(b)
    nota.octave = 2
    dur = 4
    nota.duration.quarterLength = dur
    bass_seq.append(nota)

In [10]:
sequence = [harmony_seq, bass_seq]
s = showNotes(sequence,[4,4])
s.show('musicxml')